In [1]:
import torch
import torch.nn as nn
import numpy as np

from model import BandSplitModule, BandSequenceModelingModule, MaskEstimationModule, torch_mdct, createFreqBands

In [2]:
x = torch.rand((2,2,44100*6))

In [3]:
from einops import rearrange

In [4]:
b,c,t = x.shape
wav = rearrange(x,'b c t -> (b c) t')
spec = torch_mdct(wav,window_length=2048,window_type='kbd')
spec_ = rearrange(spec, '(b c) f t -> b f t c', c=2)

# Modules
bandsplitter = BandSplitModule(sample_rate=44100,n_fft=2048,channels=2,fc_dim=128, bands=[1000, 4000, 8000, 16000, 20000], num_subBands=[10, 12, 8 , 8, 2, 1])
bandSequencer = nn.Sequential(*[BandSequenceModelingModule(channels=2, fc_dim=128, num_subBands=[10, 12, 8 , 8, 2, 1]) for _ in range(12)])
maskEstimator = MaskEstimationModule(44100,2048)

# Continue
z = bandsplitter(spec_)
q = bandSequencer(z)
mask = maskEstimator(q)

KBD Window: 2048


In [5]:
spec.shape

torch.Size([4, 1023, 260])

In [6]:
(spec*mask).shape

torch.Size([4, 1023, 260])

In [7]:
est_spec = spec*mask

In [8]:
from model import torch_imdct

In [9]:
est_wav = torch_imdct(est_spec,sample_length=6*44100,window_length=2048)

In [12]:
est_wav_rearrange = rearrange(est_wav,'(b c) t -> b c t',b=2,c=2)

In [17]:
est_wav_rearrange[:,:,:t].shape

torch.Size([2, 2, 264600])

In [19]:
est_spec.shape

torch.Size([4, 1023, 260])

In [10]:
torch_imdct(spec[0],sample_length=6*44100,window_length=2048).shape

ValueError: not enough values to unpack (expected 2, got 1)